In [6]:
import pandas as pd
!conda install -c conda-forge/label/cf201901 wikipedia
import wikipedia as wp


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wikipedia-1.4.0            |           py35_0          20 KB  conda-forge/label/cf201901

The following NEW packages will be INSTALLED:

    wikipedia: 1.4.0-py35_0 conda-forge/label/cf201901


wikipedia-1.4.0      | 20 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


## Read data from wikipedia

In [7]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
Toronto_df = pd.read_html(html)[0]
Toronto_df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


## Renaming Header

In [8]:
Toronto_df.columns=Toronto_df.iloc[0]
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [9]:
Toronto_df=Toronto_df.reindex(Toronto_df.index.drop(0))
Toronto_df.head()
Toronto_df.shape

(288, 3)

## Exclude the records where Borough=Not assigned, then reindexing the dataframe

In [10]:
Toronto_database=Toronto_df[Toronto_df['Borough']!='Not assigned']
Toronto_database=Toronto_database.reset_index()
Toronto_database=Toronto_database.drop('index',1)
Toronto_database.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Consider Borough in Neighbourhood where Neighbourhood=Not assigned

In [11]:
Toronto_database1=Toronto_database[Toronto_database['Neighbourhood']=='Not assigned']
Toronto_database1=Toronto_database1.drop('Neighbourhood',1)
Toronto_database1['Neighbourhood']=Toronto_database1['Borough']
Toronto_database1

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Queen's Park


In [12]:
Toronto_database=Toronto_df[Toronto_df['Neighbourhood']!='Not assigned']
Toronto_database=Toronto_database.append(Toronto_database1)
Toronto_database.shape

(211, 3)

## Merge Neighbourhood where based on duplicate Postcode

In [13]:

TDF=Toronto_database.groupby('Postcode').agg({
                'Borough': lambda x: x.unique(),
                'Neighbourhood': lambda x: ', '.join(x)
    })

TDF=TDF.reset_index()
TDF

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,"Birch Cliff, Cliffside West",Scarborough


## Final Table size

In [14]:
TDF.shape

(103, 3)

In [21]:
geodf=pd.read_csv('http://cocl.us/Geospatial_data')

geodf = geodf.rename(columns = {"Postal Code": "Postcode", 
                                  "Latitude":"Latitude", 
                                  "Longitude": "Longitude"}) 
geodf.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
Toronto_DB = pd.merge(TDF,
                 geodf[['Postcode','Latitude', 'Longitude']],
                 on='Postcode')

Toronto_DB=Toronto_DB[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
Toronto_DB

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
